# RNNs
We want a layer which takes in not just an input vector $x$ but a(n ordered) sequence of vectors $x_1,\ldots,x_n$, where $x_i \in R^d$.

One thing we could do is stack them on top of one another, to create a big vector $\vec x \in R^{nd}$, and then apply a fully connected layer.
But we want to be smarter than that.

* An RNNcell is just a function $f(x,h)$ taking two inputs: the first is the input vector $x \in R^d$, the second is called the *hidden* vector $h \in R^k$.
* The output of $y = f(x,h) \in R^k$ has the same size as the hidden vector.
* An RNN is built by stacking RNNcells.
* Say the input is the sequence $(x_1,\ldots,x_n)$ with $x_i \in R^d$. We proceed step by step.
    * $h_0 := \vec 0$ (this can also be initialized differently)
    * $h_1 := f(x_1,h_0)$
    * $h_2 := f(x_2,h_1)$
    * $h_3 := f(x_3,h_2)$
    * ...
    * $h_n := f(x_n,h_{n-1})$
* The output of the RNN is the whole sequence $(h_1,\ldots,h_n)$.


## Let's see this in action using `pytorch`.

In [1]:
import torch
import torch.nn as nn

### We start by fixing n,k,d from above.
(since it's pytorch, we must also work in batches)

In [2]:
# n = seq_len = 15
seq_len = 15
# d = input_size = 7
input_size = 7
# k = hidden_size = 5
hidden_size = 5
# batch_len (aka batch_size) = 4
batch_len = 4

### We now declare the RNN layer.

In [3]:
rnnlayer = nn.GRU(input_size=input_size,
                 hidden_size=hidden_size,
                 num_layers=1,
                 bias=True,
                 batch_first=True,
                 dropout=0,
                 bidirectional=False)

## Let's go over the parameters:

- `input_size` and `hidden_size` we know about
- `num_layers` says how many cells should be stacked on top of one another (the default is 1) [let's ignore this]
- `bias` is the usual: adding a constant term (default = True).
- `batch_first` decides if `input.shape` is (batch_size,seq_len,input_size) or (seq_len,batch_size,input_size). Same for outputs.
    - `batch_first` is FALSE by default!
- `dropout` is dropout, default = 0 [let's ignore this]
- `bidirectional` is if we want a bidirectional RNN, default = False. [let's ignore this]

#### Let's define some bogus inputs.

In [9]:
# bogus input
x = torch.randn(batch_len,seq_len,input_size)

In [10]:
x.shape

torch.Size([4, 15, 7])

## Inputs
If we look closely, `rnnlayer` actually takes two inputs:

- the first is the squence of input vectors, whose shape is (batch_len, seq_len, input_size).
- the second is zero-th hidden vector, of shape: (1, batch_len, hidden_size).

The reason for that funny 1 is that the real shape is (num_layers*num_directions, batch_len, hidden_size)
- for us `num_layers = 1`, `num_directions = 1` (as bidirectional=False).

By default, the second input is 0.

## Let's have a look at the output.

In [6]:
# output is a tuple
H, h_n = rnnlayer(x)

- `H` is the collection of all hidden vectors, with shape (batch_len, seq_len, hidden_size).

(the actual shape is (batch_len, seq_len, num_directions*hidden_size), but num_directions=1 for us)

In [7]:
H.shape

torch.Size([4, 15, 5])

- `h_n` is the output of the last cell
- `h_n` has shape (1,batch_len, hidden_size)

The true shape is (num_layers$*$num_directions, batch_len, hidden_size), but num_layers$*$num_directions = 1 for us.
- if you have more than layer, this allows you to separate the output from each, which may be useful

In [21]:
h_n.shape

torch.Size([1, 4, 5])

In [23]:
# and we can always squeeze out the zeroth index
h_n.squeeze(dim=0).shape

torch.Size([4, 5])

## Bidirectional
Bidirectional RNNs are a slight variant of normal RNNs, which are super useful in sequence-to-sequence models. In this repository we take individual characters to be tokens, and words (in our case, names) to be sequences. But if we wanted instead to train a machine to translate between two languages, we'd be better off having tokens be entire words, and sequences sentences. Rao's book gives the example of the sentence 'A man who hunts ducks oout on weekend.' A unidirectional RNN would read the sentence from left to right, and would interpret ducks as a noun. However, once we've read the whole sentence, we would actualy interpret ducks as a verb. It would be useful to have a model which could read sentences both ways! This is actually easily achieved, simply by working with ***bidirectional RNNs***, which are nothing but two parallel RNNs running at the same time. Let's explain how.

Let $f(x,h)$ denote a single RNN cell, as before. Suppose our sequence consists of vectors $(x_1,\ldots,x_l)$. In one direction, we run our standard RNN, which takes as an input a hidden vector $h_0$, and spits out a final $h_n$. Recall this is done ieratively as: 
- $h_1 = f(x_1,h_0)$
- $h_2 = f(x_2,h_1)$
- ...
- $h_n = f(x_n,h_{n-1}$

At the same time, we have another RNN, taking as input a hidden vector $k_0$ and spitting out a final $k_n$. These are defined iteratively as: 
- $k_1 = f(x_n,k_0$
- $k_2 = f(x_{n-1},k_1)$
- $k_3 = f(x_{n-2},k_2)$
- ...
- $k_n = f(x_1,k_{n-1})$

The vectors $h_n$, $k_n$ are then concatenated to produce the total output of the bidirectional RNN.
Let's implement one!

In [25]:
import torch
batch_len = 4
seq_len = 15
input_size = 7
hidden_size = 5
birnnlayer = nn.GRU(input_size=input_size,
                 hidden_size=hidden_size,
                 num_layers=1,
                 bias=True,
                 batch_first=True,
                 dropout=0,
                 bidirectional=True)

x = torch.randn(batch_len,seq_len,input_size)
H, h = birnnlayer(x)

In [26]:
H.shape

torch.Size([4, 15, 10])

Here `H` is the collection of all hidden vectors.
- 4 is the batch length
- 15 is the sequence length
- 10 = hidden_size $*$ num_directions = 5 $*$ 2
So the last component is stacking the hidden outputs from both directions on top of one another.

In [27]:
h.shape

torch.Size([2, 4, 5])

Here `h` is doing something slightly different.
- 4 is the batch length
- 5 is hidden size
- 2 = num_directions
So `h` just stacks the final output vectors from both directions, along a new dummy index.

In [30]:
h_f = h[0]
h_b = h[1]
print(h_f.shape, h_b.shape)

torch.Size([4, 5]) torch.Size([4, 5])


## Packing
Since sequences tend to be of different lengths, there's always a need for padding. Instead of looking up in advance the length of the longest sequence, we can work batch by batch. Packed sequences are an efficient way to do that. That's it.

``` Note: A sequence must be sorted by length before being packed. ```

For the example below assume our vocabulary maps letters to indices alphabetically. We will pack the batch consisting of sequences 'abcd', 'efg,', 'h'.

The example is taken from here: https://github.com/joosthub/PyTorchNLPBook/blob/master/chapters/chapter_8/8_PackedSequence_example.ipynb.

In [36]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
abcd_padded = torch.tensor([1,2,3,4], dtype=torch.float32)
efg_padded = torch.tensor([5,6,7,0], dtype=torch.float32)
h_padded = torch.tensor([8,0,0,0], dtype=torch.float32)

padded_tensor = torch.stack([abcd_padded, efg_padded, h_padded])

padded_tensor

tensor([[1., 2., 3., 4.],
        [5., 6., 7., 0.],
        [8., 0., 0., 0.]])

In [35]:
lengths = [4,3,1]

packed_tensor = pack_padded_sequence(padded_tensor, lengths, batch_first=True)

packed_tensor

PackedSequence(data=tensor([1., 5., 8., 2., 6., 3., 7., 4.]), batch_sizes=tensor([3, 2, 2, 1]), sorted_indices=None, unsorted_indices=None)

In [38]:
unpacked_tensor, unpacked_lengths = pad_packed_sequence(packed_tensor, batch_first=True)
print(lengths)
unpacked_tensor

[4, 3, 1]


tensor([[1., 2., 3., 4.],
        [5., 6., 7., 0.],
        [8., 0., 0., 0.]])